# Football Player Value
- Data From : https://www.transfermarkt.com/
    * 트랜스퍼마켓 : 공신력 있는 축구선수 이적 관련 사이트
- Subject : 축구선수 몸값은 어떻게 결정될까?
    * 1. 축구가 산업화된 잉글랜드 프리미어리그에선 영국 국적이 몸값에 영향을 준다고 한다. 정말?
    * 2. 골, 어시스트, 키, 몸무게, 국적, 소속팀, 소속팀 성적, 소속팀 소속 리그 등 다양한 변수 활용
- Project Process
    * 1. 웹크롤링으로 데이터 확보
    * 2. 확보한 데이터 병합
    * 3. EDA 및 데이터전처리
    * 4. 서브주제에 맞춰 다양한 ML모델 사용
- Stack : Beautiful Soup 4, Pandas, Seaborn, Scikit-Learning

In [14]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import re
# 정규표현식

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV # 데이터 나누기, 점검 및 훈련
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_score, recall_score # 평가지표
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR, LinearSVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.pipeline import make_pipeline
import multiprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import time

## 1. 축구선수 몸값 데이터 확보
- webcrawling
- transfer markt

In [17]:
# 페이지 넘어가는 url 찾는 법
# 페이지 넘기는 링크(버튼) 우클릭 -> 링크 복사 -> 붙여넣기 -> url에 page= 있음. -> 여기에 page값 입력해 확인해보기
url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={}".format(1)
r = requests.get(url, headers=headers)
r.status_code

200

In [18]:
soup = BeautifulSoup(r.text, "html.parser")

In [19]:
player = soup.find_all('tr', class_=['odd', 'even'])
player

[<tr class="odd">
 <td class="zentriert">1</td><td class=""><table class="inline-table"><tr><td rowspan="2"><a href="#"><img alt="Kylian Mbappé" class="bilderrahmen-fixed" src="https://img.a.transfermarkt.technology/portrait/small/342229-1632686046.jpg?lm=1" title="Kylian Mbappé"/></a></td><td class="hauptlink"><a href="/kylian-mbappe/profil/spieler/342229" title="Kylian Mbappé">Kylian Mbappé</a></td></tr><tr><td>Centre-Forward</td></tr></table></td><td class="zentriert">23</td><td class="zentriert"><img alt="France" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/verysmall/50.png?lm=1520611569" title="France"/></td><td class="zentriert"><a href="/fc-paris-saint-germain/startseite/verein/583" title="Paris Saint-Germain"><img alt="Paris Saint-Germain" class="" src="https://tmssl.akamaized.net/images/wappen/verysmall/583.png?lm=1522312728" title=" "/></a></td><td class="rechts hauptlink"><a href="/kylian-mbappe/marktwertverlauf/spieler/342229">€160.00m</a> </td><td c

In [24]:
data = player[0].find_all('td')
data

[<td class="zentriert">1</td>,
 <td class=""><table class="inline-table"><tr><td rowspan="2"><a href="#"><img alt="Kylian Mbappé" class="bilderrahmen-fixed" src="https://img.a.transfermarkt.technology/portrait/small/342229-1632686046.jpg?lm=1" title="Kylian Mbappé"/></a></td><td class="hauptlink"><a href="/kylian-mbappe/profil/spieler/342229" title="Kylian Mbappé">Kylian Mbappé</a></td></tr><tr><td>Centre-Forward</td></tr></table></td>,
 <td rowspan="2"><a href="#"><img alt="Kylian Mbappé" class="bilderrahmen-fixed" src="https://img.a.transfermarkt.technology/portrait/small/342229-1632686046.jpg?lm=1" title="Kylian Mbappé"/></a></td>,
 <td class="hauptlink"><a href="/kylian-mbappe/profil/spieler/342229" title="Kylian Mbappé">Kylian Mbappé</a></td>,
 <td>Centre-Forward</td>,
 <td class="zentriert">23</td>,
 <td class="zentriert"><img alt="France" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/verysmall/50.png?lm=1520611569" title="France"/></td>,
 <td class="zentri

In [32]:
ranking = data[0].text
print(ranking)
name = data[3].text
print(name)
position = data[4].text
print(position)
nation = data[6].img['alt'] #data[6].title로 입력해도 된다.
print(nation)
club = data[7].img['alt']
print(club)
value = data[8].text.strip()
print(value)
matches = data[9].text
print(matches)
goal = data[10].text
print(goal)
assist = data[12].text
print(assist)
yellow = data[13].text
print(yellow)
red = data[15].text
print(red)
subsin = data[16].text
print(subsin)
subsout = data[17].text
print(subsout)

1
Kylian Mbappé
Centre-Forward
France
Paris Saint-Germain
€160.00m
37
26
17
9
0
2
12


In [33]:
## 반복문 이용해서 모든 페이지에 있는 데이터 긁어오기

ranking = []
name = []
position = []
age = []
nation = []
club = []
value = []
matches = []
goals = []
assist = []
yellowcard = []
redcard = []
subsin = []
subsout = []

headers = {'User-Agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}


# 범위를 지정할 땐 마지막 다음 순서까지 입력. 1부터 5 직전까지. #따라서 모든 행위는 반복문 아래에서 진행되도록 해야한다. tap키 눌러주세요.
for i in range(1, 21):
    url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={}".format(i)
    # url = f"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page={i}"
    # f ~ {i}는 변수처리를 의미한다.
    # 반복문을 통해 원하는 페이지까지 스크롤링하도록 i값을 메기면 된다. 
    # url = f"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page={i}"
    r = requests.get(url, headers=headers)
    r.status_code

    soup = BeautifulSoup(r.text, 'html.parser')
    # 여기까지가 뷰티풀숲 사용하기 위한 준비

    player_info = soup.find_all('tr', class_=['odd', 'even'])
    # 데이터 구조가 1, 2페이지 같은지 확인 한번 해보기!

    for i in player_info:
        data = i.find_all('td')
        ranking.append(int(data[0].text)) #앞 장에서 ranking과 age가 object형임을 확인. int로 바꿔서 리스트에 넣어줘야합니다. 그래서 int()로 감싸 자료들 형태를 바꿔줘야해요.
        name.append(data[3].text)
        position.append(data[4].text)
        age.append(int(data[5].text))
        nation.append(data[6].img['alt'])
        club.append(data[7].img['alt'])
        value.append(data[8].text.strip()) #replace, astype=('float')으로
        matches.append(data[9].text)
        goals.append(data[10].text)
        assist.append(data[12].text)
        yellowcard.append(data[13].text)
        redcard.append(data[15].text)
        subsin.append(data[16].text)
        subsout.append(data[17].text)
        

time.sleep(2)

# 데이터프레임을 만드는 과정엔 반복이 필요하지 않으니 나와줘야합니다.
# 반복은 페이지에서 정보를 빼오고 이를 리스트로 만드는 과정까지만
# 반복문을 사용할 땐 정확히 어느 과정까지 반복이 필요한지 명확히 해놔야 편합니다.
df = pd.DataFrame(
    {"ranking":ranking,
    "name":name,
    "position":position,
    "age":age,
    "nation":nation,
    "club":club,
    "value":value,
    "matches":matches,
    "goals":goals,
    "assist":assist,
    "yellowcard":yellowcard,
    "redcard":redcard,
    "subsin":subsin,
    "subsout":subsout})

df

,ranking,name,position,age,nation,club,value,matches,goals,assist,yellowcard,redcard,subsin,subsout
0,1,Kylian Mbappé,Centre-Forward,23,France,Paris Saint-Germain,€160.00m,37,26,17,9,0,2,12
1,2,Erling Haaland,Centre-Forward,21,Norway,Borussia Dortmund,€150.00m,24,24,6,3,0,4,6
2,3,Vinicius Junior,Left Winger,21,Brazil,Real Madrid,€100.00m,39,17,14,7,0,2,21
3,4,Mohamed Salah,Right Winger,29,Egypt,Liverpool FC,€100.00m,43,30,11,1,0,2,10
4,5,Harry Kane,Centre-Forward,28,England,Tottenham Hotspur,€100.00m,42,23,6,3,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,Adrien Rabiot,Central Midfield,26,France,Juventus FC,€18.00m,37,0,2,5,0,7,10
496,497,Gaëtan Laborde,Centre-Forward,27,France,Stade Rennais FC,€18.00m,38,19,7,4,0,2,17
497,498,Alex Oxlade-Chamberlain,Central Midfield,28,England,Liverpool FC,€18.00m,29,3,2,0,0,12,13
498,499,Daniel Carvajal,Right-Back,30,Spain,Real Madrid,€18.00m,27,1,2,2,0,4,14


## 2. Club Data
- 소속리그 칼럼 추가하기 위해 추가로 데이터 긁어오겠습니다.
- club data를 앞서 확보한 player에 merge 해줄 예정

In [39]:
url = f"https://www.transfermarkt.com/vereins-statistik/wertvollstemannschaften/marktwertetop?ajax=yw1&kontinent_id=0&land_id=0&page={1}&plus=1"
r = requests.get(url, headers=headers)
r.status_code

200

In [40]:
soup = BeautifulSoup(r.text, "html.parser")

In [41]:
# 소스코드 확인결과 플레이어와 같았습니다. 그래서 그대로 진행
team = soup.find_all('tr', class_=['odd', 'even'])
team

[<tr class="odd">
 <td class="zentriert">1</td><td class="zentriert no-border-rechts"><a href="/manchester-city/startseite/verein/281/saison_id/2021" title="Manchester City"><img alt="Manchester City" class="tiny_wappen" src="https://tmssl.akamaized.net/images/wappen/tiny/281.png?lm=1467356331" title=" "/></a></td><td class="no-border-links hauptlink"><a href="/manchester-city/startseite/verein/281" title="Manchester City">Manchester City</a></td><td class="links"><img alt="England" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/tiny/189.png?lm=1520611569" title="England"/> <a href="/wettbewerb/startseite/wettbewerb/GB1" title="Premier League">Premier League</a></td><td class="zentriert">22</td><td class="zentriert">28.0</td><td class="rechts"><b>€977.30m</b><span class="icons_sprite red-arrow-ten"> </span></td><td class="rechts"><b>€44.42m</b></td><td class="rechts">€962.00m</td><td class="rechts">98.4 %</td></tr>,
 <tr class="even">
 <td class="zentriert">2</td>

In [42]:
mc = team[0].find_all('a')
mc

[<a href="/manchester-city/startseite/verein/281/saison_id/2021" title="Manchester City"><img alt="Manchester City" class="tiny_wappen" src="https://tmssl.akamaized.net/images/wappen/tiny/281.png?lm=1467356331" title=" "/></a>,
 <a href="/manchester-city/startseite/verein/281" title="Manchester City">Manchester City</a>,
 <a href="/wettbewerb/startseite/wettbewerb/GB1" title="Premier League">Premier League</a>]

In [48]:
name = mc[1].text
name

'Manchester City'

In [50]:
league = mc[2].text
league

'Premier League'

In [111]:
team = []
league = []

headers = {'User-Agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}


# 범위를 지정할 땐 마지막 다음 순서까지 입력. 1부터 5 직전까지. #따라서 모든 행위는 반복문 아래에서 진행되도록 해야한다. tap키 눌러주세요.
for i in range(1, 21):
    url = f"https://www.transfermarkt.com/vereins-statistik/wertvollstemannschaften/marktwertetop?ajax=yw1&kontinent_id=6&land_id=0&page={i}&plus=1"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 여기까지가 뷰티풀숲 사용하기 위한 준비

    team_info = soup.find_all('tr', class_=['odd', 'even'])
    # 데이터 구조가 1, 2페이지 같은지 확인 한번 해보기!

    for i in team_info:
        data = i.find_all('a')
        team.append(data[1].text) #앞 장에서 ranking과 age가 object형임을 확인. int로 바꿔서 리스트에 넣어줘야합니다. 그래서 int()로 감싸 자료들 형태를 바꿔줘야해요.
        league.append(data[2].text)
        

time.sleep(2)

# 데이터프레임을 만드는 과정엔 반복이 필요하지 않으니 나와줘야합니다.
# 반복은 페이지에서 정보를 빼오고 이를 리스트로 만드는 과정까지만
# 반복문을 사용할 땐 정확히 어느 과정까지 반복이 필요한지 명확히 해놔야 편합니다.
team_df = pd.DataFrame(
    {"club":team,
    "league":league})

team_df

,club,league
0,Manchester City,Premier League
1,Paris Saint-Germain,Ligue 1
2,Liverpool FC,Premier League
3,Chelsea FC,Premier League
4,Bayern Munich,Bundesliga
...,...,...
495,Sheffield United,Championship
496,FC Augsburg,Bundesliga
497,CA Osasuna,LaLiga
498,GNK Dinamo Zagreb,1.HNL


In [112]:
data = pd.merge(df, team_df, left_on = 'club', right_on='club', how='outer')
data

,ranking,name,position,age,nation,club,value,matches,goals,assist,yellowcard,redcard,subsin,subsout,league
0,1.0,Kylian Mbappé,Centre-Forward,23.0,France,Paris Saint-Germain,€160.00m,37,26,17,9,0,2,12,Ligue 1
1,9.0,Neymar,Left Winger,30.0,Brazil,Paris Saint-Germain,€90.00m,21,5,5,5,0,1,8,Ligue 1
2,26.0,Marquinhos,Centre-Back,27.0,Brazil,Paris Saint-Germain,€75.00m,32,3,1,3,0,0,0,Ligue 1
3,31.0,Achraf Hakimi,Right-Back,23.0,Morocco,Paris Saint-Germain,€70.00m,37,5,5,7,1,3,4,Ligue 1
4,46.0,Gianluigi Donnarumma,Goalkeeper,23.0,Italy,Paris Saint-Germain,€65.00m,20,0,0,3,0,0,0,Ligue 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,NaN,NaN,NaN,NaN,NaN,Hertha BSC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bundesliga
931,NaN,NaN,NaN,NaN,NaN,Hertha BSC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bundesliga
932,NaN,NaN,NaN,NaN,NaN,Hertha BSC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bundesliga
933,NaN,NaN,NaN,NaN,NaN,Hertha BSC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bundesliga


In [113]:
# 중복 제거
data.drop_duplicates(['name'], keep='first', inplace=True, ignore_index=False)
data.sort_values(by=['ranking'], inplace=True)
data

,ranking,name,position,age,nation,club,value,matches,goals,assist,yellowcard,redcard,subsin,subsout,league
0,1.0,Kylian Mbappé,Centre-Forward,23.0,France,Paris Saint-Germain,€160.00m,37,26,17,9,0,2,12,Ligue 1
15,2.0,Erling Haaland,Centre-Forward,21.0,Norway,Borussia Dortmund,€150.00m,24,24,6,3,0,4,6,Bundesliga
25,3.0,Vinicius Junior,Left Winger,21.0,Brazil,Real Madrid,€100.00m,39,17,14,7,0,2,21,LaLiga
39,4.0,Mohamed Salah,Right Winger,29.0,Egypt,Liverpool FC,€100.00m,43,30,11,1,0,2,10,Premier League
55,5.0,Harry Kane,Centre-Forward,28.0,England,Tottenham Hotspur,€100.00m,42,23,6,3,0,3,4,Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,497.0,Gaëtan Laborde,Centre-Forward,27.0,France,Stade Rennais FC,€18.00m,38,19,7,4,0,2,17,Ligue 1
54,498.0,Alex Oxlade-Chamberlain,Central Midfield,28.0,England,Liverpool FC,€18.00m,29,3,2,0,0,12,13,Premier League
38,499.0,Daniel Carvajal,Right-Back,30.0,Spain,Real Madrid,€18.00m,27,1,2,2,0,4,14,LaLiga
643,500.0,João Pedro,Second Striker,30.0,Italy,Cagliari Calcio,€18.00m,32,10,4,8,0,2,2,Serie A


In [114]:
data.reset_index(inplace=True)
del data['index']
data

,ranking,name,position,age,nation,club,value,matches,goals,assist,yellowcard,redcard,subsin,subsout,league
0,1.0,Kylian Mbappé,Centre-Forward,23.0,France,Paris Saint-Germain,€160.00m,37,26,17,9,0,2,12,Ligue 1
1,2.0,Erling Haaland,Centre-Forward,21.0,Norway,Borussia Dortmund,€150.00m,24,24,6,3,0,4,6,Bundesliga
2,3.0,Vinicius Junior,Left Winger,21.0,Brazil,Real Madrid,€100.00m,39,17,14,7,0,2,21,LaLiga
3,4.0,Mohamed Salah,Right Winger,29.0,Egypt,Liverpool FC,€100.00m,43,30,11,1,0,2,10,Premier League
4,5.0,Harry Kane,Centre-Forward,28.0,England,Tottenham Hotspur,€100.00m,42,23,6,3,0,3,4,Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,497.0,Gaëtan Laborde,Centre-Forward,27.0,France,Stade Rennais FC,€18.00m,38,19,7,4,0,2,17,Ligue 1
497,498.0,Alex Oxlade-Chamberlain,Central Midfield,28.0,England,Liverpool FC,€18.00m,29,3,2,0,0,12,13,Premier League
498,499.0,Daniel Carvajal,Right-Back,30.0,Spain,Real Madrid,€18.00m,27,1,2,2,0,4,14,LaLiga
499,500.0,João Pedro,Second Striker,30.0,Italy,Cagliari Calcio,€18.00m,32,10,4,8,0,2,2,Serie A


In [115]:
data[data['league'].isna() == True]

,ranking,name,position,age,nation,club,value,matches,goals,assist,yellowcard,redcard,subsin,subsout,league
259,260.0,Gabriel Barbosa,Centre-Forward,25.0,Brazil,Clube de Regatas do Flamengo,€26.00m,11,9,1,4,0,1,2,NaN
325,326.0,Julián Álvarez,Centre-Forward,22.0,Argentina,CA River Plate,€23.00m,7,5,0,0,0,0,2,NaN
377,378.0,Thiago Almada,Attacking Midfield,20.0,Argentina,Atlanta United FC,€20.00m,3,1,0,0,0,3,0,NaN
379,380.0,Pape Sarr,Defensive Midfield,19.0,Senegal,FC Metz,€20.00m,26,1,1,6,0,10,5,NaN
446,447.0,Adam Hlozek,Second Striker,19.0,Czech Republic,AC Sparta Prague,€19.00m,40,9,13,3,0,2,11,NaN
448,449.0,Danilo,Defensive Midfield,20.0,Brazil,Sociedade Esportiva Palmeiras,€18.00m,10,1,0,1,0,1,1,NaN
471,472.0,Umar Sadiq,Centre-Forward,25.0,Nigeria,UD Almería,€18.00m,33,17,10,5,0,6,13,NaN


In [117]:
df.to_csv("data/df.csv", encoding='utf-8')
team_df.to_csv("data/team_df.csv", encoding='utf-8')
data.to_csv("data/data.csv", encoding='utf-8')